In [50]:
import pandas as pd

df = pd.DataFrame(pd.DataFrame({
                    'date',
                    'minute',
                    'match_id',
                    'homeTeam',
                    'awayTeam',
                    'goalHome',
                    'goalAway',
                    'shotsHome',
                    'shotsAway',
                    'crossesHome',
                    'crossesAway',
                    'assistsHome',
                    'assistsAway',
                    'blockedShotsHome',
                    'blockedShotsAway',
                    'league',
                    'corners_home',
                    'corners_away',
                    'redcards_home',
                    'redcards_away',
                    'shotsOffgoal_home',
                    'shotsOffgoal_away',
                    'shotsOngoal_home',
                    'shotsOngoal_away',
                    'yellowcards_home',
                    'yellowcards_away',
                    'passes_home',
                    'passes_away',
                    'fouls_c_home',
                    'fouls_c_away',
                    'fouls_won_home',
                    'fouls_won_away',
                    'offsides_home',
                    'offsides_away',
                    'tackles_home',
                    'tackles_away',

                }))

df.to_csv('data_live_scores_full.csv', index=False, encoding='utf-8')

In [135]:
from io import StringIO
import traceback
import datetime
from concurrent.futures import ThreadPoolExecutor
import os
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# file_path = './data_live_scores_full.csv'

def main_function(date_str):

    try:
        # df = pd.read_csv(file_path, encoding='utf-8')
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        # options = webdriver.ChromeOptions()
        # driver = webdriver.Remote(command_executor='http://localhost:4444', options=options)
        time.sleep(1)


        print(f'Verificando jogos da data: {date_str}')

        # Acessa o site
        driver.get(
            f'https://optaplayerstats.statsperform.com/en_GB/soccer?date={date_str}')
        driver.maximize_window()
        

        # Espera a página carregar
        WebDriverWait(driver, 30).until(lambda d: d.execute_script(
            'return document.readyState') == 'complete')
        
        close_icon = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable(
                (By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
        )
        close_icon.click()
        # Obtém o conteúdo da página
        page_content = driver.page_source
        # Analisa o conteúdo com BeautifulSoup
        soup = BeautifulSoup(page_content, 'html.parser')

        # Encontrar todos os blocos de competição
        competitions = soup.find_all(
            'div', class_='livescore-container-fixtures-competition')
        dict_league_links = {}

        for comp in competitions:
            # Dentro de cada bloco de competição, obter o nome da liga
            league_name = comp.find(
                'span', class_='competitionName').get_text(strip=True)

            # Obter todos os links relacionados a essa liga
            links = comp.find_all('a', href=True)
            for link in links:
                if 'match' in link['href']:
                    if league_name not in dict_league_links:
                        dict_league_links[league_name] = []
                        dict_league_links[league_name].append(
                            'https://optaplayerstats.statsperform.com' + link['href'])
                    else:
                        dict_league_links[league_name].append(
                            'https://optaplayerstats.statsperform.com' + link['href'])
    except Exception as e:
        print(f'Erro na data: {date_str}')
        print(e)
        pass

    colunas_ordenadas =  ['date', 'minute', 'homeTeam', 'awayTeam', 'goalHome', 'goalAway',
                           'shotsHome', 'shotsAway', 'crossesHome', 'crossesAway','assistsHome', 'assistsAway', 
                           'blockedShotsHome', 'blockedShotsAway', 'league', 'corners_home',
                            'corners_away', 'redcards_home', 'redcards_away', 'shotsOffgoal_home',
                            'shotsOffgoal_away', 'shotsOngoal_home', 'shotsOngoal_away',
                            'yellowcards_home', 'yellowcards_away', 'passes_home', 'passes_away',
                            'fouls_c_home', 'fouls_c_away', 'fouls_won_home', 'fouls_won_away',
                            'offsides_home', 'offsides_away', 'tackles_home', 'tackles_away',
                            'result', 'match_id']
    
    for league in dict_league_links:
        print("Existem " + str(len(dict_league_links[league])) + " jogos para a liga " + league)
        for links in dict_league_links[league]:
            try:
                flag = 0

                driver.get(links)
                button = WebDriverWait(driver, 30).until(
                    EC.element_to_be_clickable(
                        (By.CSS_SELECTOR, 'button[data-period_idx="1"].Opta-Stdperiod'))
                )

                home_team = driver.find_element(
                    By.XPATH, '//*[@id="Opta_2"]/div/div/div[1]/div/ul/li[2]/a').text
                away_team = driver.find_element(
                    By.XPATH, '//*[@id="Opta_2"]/div/div/div[1]/div/ul/li[3]').text
                match_id = date_str + '-' + home_team + '-' + away_team

                # print('Verificando se o jogo já existe no dataframe')

                # if match_id in df['match_id'].unique():
                #     print('Jogo já existe no dataframe')
                #     continue
                
                # print('Jogo não existe no dataframe')
                # print('Iniciando a coleta de dados')
                # print('-------------------')

                # Clica no botão "First half"
                # button.click()

                # Recupera o valor do 'transform' após o clique
                # dragger = driver.find_element(
                #     By.CSS_SELECTOR, '.Opta-Dragger.Opta-Dragger-end')

                # transform_value = dragger.get_attribute('transform')

                # # Usa regex para extrair o valor numérico de 'translate'
                # match = re.search(r'translate\((\d+),', transform_value)
                # # Se não encontrar, usa um valor padrão
                # end_value = int(match.group(1)) if match else 35

                # Use ActionChains para simular o arrasto
                # actions = ActionChains(driver)

                # Exemplo: movendo o dragger 5 pixels à direita
                # Aqui estamos variando de 30 a 35
                # team_tab_xpath1 = '//*[@id="Opta_2"]/div/div/div[1]/div/ul/li[2]/a'
                # team_tab_xpath2 = '//*[@id="Opta_2"]/div/div/div[1]/div/ul/li[3]/a'
                # driver.find_element(By.XPATH, team_tab_xpath1).click()

                date = []
                minute = []
                match_id_list = []
                homeTeam = []
                awayTeam = []
                goalHome = []
                goalAway = []
                shotsHome = []
                shotsAway = []
                crossesHome = []
                crossesAway = []
                assistsHome = []
                assistsAway = []
                blockedShotsHome = []
                blockedShotsAway = []
                liga = []
                corners_home = []
                corners_away = []
                redcards_home = []
                redcards_away = []
                shotsOffgoal_home = []
                shotsOffgoal_away = []
                shotsOngoal_home = []
                shotsOngoal_away = []
                yellowcards_home = []
                yellowcards_away = []
                passes_home = []
                passes_away = []
                fouls_c_home = []
                fouls_c_away = []
                fouls_won_home = []
                fouls_won_away = []
                offsides_home = []
                offsides_away = []
                tackles_home = []
                tackles_away = []
                result = []

                dragger = driver.find_element(
                        By.CSS_SELECTOR, '.Opta-Dragger.Opta-Dragger-end')

                dragger_location = dragger.location
                start_x = dragger_location['x']
                start_y = dragger_location['y']

                for value in range(0, start_x, 5):
                    all_dataframes = {}
                    # Localiza o elemento e altera seu valor de transform
                    dragger = driver.find_element(
                        By.CSS_SELECTOR, '.Opta-Dragger.Opta-Dragger-end')

                    # driver.execute_script(
                    #     f"arguments[0].setAttribute('transform', 'translate({value}, 45)')", dragger)
                    # actions.click_and_hold(dragger).move_by_offset(
                    #     4, 0).release().perform()

                    # Obter a posição atual do dragger
                    dragger_location = dragger.location
                    start_x = dragger_location['x']
                    start_y = dragger_location['y']

                    print(f'Valor atual do dragger: {start_x}')

                    # Calcular a nova posição X
                    new_x = start_x - 5  # Ajuste este valor conforme necessário
                    new_y = start_y  # Mantendo a posição Y constante

                    print(f'Novo valor do dragger: {new_x}')

                    # Criar e executar o script para simular o movimento do mouse
                    script = f"""
                        var dragger = document.querySelector('#Opta_2 > div > div > div.Opta-timecontrols > div > div.Opta-Timestrip > svg > g.Opta-Dragger.Opta-Dragger-end');
                        var startX = {start_x};
                        var startY = {start_y};
                        var endX = {new_x};
                        var endY = {new_y};

                        // Criar e disparar o evento mousedown
                        var mouseDownEvent = new MouseEvent('mousedown', {{
                            clientX: startX,
                            clientY: startY,
                            bubbles: true,
                            cancelable: true,
                            view: window
                        }});
                        dragger.dispatchEvent(mouseDownEvent);

                        // Criar e disparar o evento mousemove
                        var mouseMoveEvent = new MouseEvent('mousemove', {{
                            clientX: endX,
                            clientY: endY,
                            bubbles: true,
                            cancelable: true,
                            view: window
                        }});
                        dragger.dispatchEvent(mouseMoveEvent);

                        // Criar e disparar o evento mouseup
                        var mouseUpEvent = new MouseEvent('mouseup', {{
                            clientX: endX,
                            clientY: endY,
                            bubbles: true,
                            cancelable: true,
                            view: window
                        }});
                        dragger.dispatchEvent(mouseUpEvent);
                    """
                    driver.execute_script(script)
                    


                    page_source = driver.page_source

                    dataframes = pd.read_html(StringIO(page_source))

                    all_dataframes[f'Team_1'] = dataframes

                    script = """
                    return document.querySelector('g.Opta-TimeBox.Opta-end text').textContent;
                    """

                    if driver.execute_script(script) == '0:00':
                        break

                    minute.append(driver.execute_script(script))

                    team1_stats = all_dataframes['Team_1'][2]
                    team2_stats = all_dataframes['Team_1'][3]

                    # dfs.append(team1_stats.tail(1))
                    # dfs.append(team2_stats.tail(1))

                    date.append(date_str)
                    match_id_list.append(match_id)
                    homeTeam.append(home_team)
                    awayTeam.append(away_team)
                    liga.append(league)
                    goalHome.append(team1_stats['G'].iloc[-1])
                    goalAway.append(team2_stats['G'].iloc[-1])
                    shotsHome.append(team1_stats['S'].iloc[-1])
                    shotsAway.append(team2_stats['S'].iloc[-1])
                    crossesHome.append(team1_stats['Crn'].iloc[-1])
                    crossesAway.append(team2_stats['Crn'].iloc[-1])
                    assistsHome.append(team1_stats['A'].iloc[-1])
                    assistsAway.append(team2_stats['A'].iloc[-1])
                    blockedShotsHome.append(team1_stats['BS'].iloc[-1])
                    blockedShotsAway.append(team2_stats['BS'].iloc[-1])

                    corners_home.append(team1_stats['Crn'].iloc[-1])
                    redcards_home.append(team1_stats['RC'].iloc[-1])
                    shotsOffgoal_home.append(team1_stats['S'].iloc[-1] - team1_stats['SOnT'].iloc[-1])
                    shotsOngoal_home.append(team1_stats['SOnT'].iloc[-1])
                    yellowcards_home.append(team1_stats['YC'].iloc[-1])
                    passes_home.append(team1_stats['P'].iloc[-1])
                    fouls_c_home.append(team1_stats['FC'].iloc[-1])
                    fouls_won_home.append(team1_stats['FW'].iloc[-1])
                    offsides_home.append(team1_stats['O'].iloc[-1])
                    tackles_home.append(team1_stats['Tk'].iloc[-1])

                    corners_away.append(team2_stats['Crn'].iloc[-1])
                    redcards_away.append(team2_stats['RC'].iloc[-1])
                    shotsOffgoal_away.append(team2_stats['S'].iloc[-1] - team2_stats['SOnT'].iloc[-1])
                    shotsOngoal_away.append(team2_stats['SOnT'].iloc[-1])
                    yellowcards_away.append(team2_stats['YC'].iloc[-1])
                    passes_away.append(team2_stats['P'].iloc[-1])
                    fouls_c_away.append(team2_stats['FC'].iloc[-1])
                    fouls_won_away.append(team2_stats['FW'].iloc[-1])
                    offsides_away.append(team2_stats['O'].iloc[-1])
                    tackles_away.append(team2_stats['Tk'].iloc[-1])
                    print('-------------------')

                    # result.append(0)

                    # goal_condition_team1 = team1_stats['G'].iloc[-1]
                    # goal_condition_team2 = team2_stats['G'].iloc[-1]

                    # if goal_condition_team1 == 1 or goal_condition_team2 == 1:
                    #     print('Gol')
                    #     # Adicione o valor 1 para os últimos 10 registros
                    #     if len(result) > 10:
                    #         result[-10:] = [1] * 10
                    #     else:
                    #         result = [1] * len(result)

                    #     break
                    print('-------------------')
                    print(f'{date_str} - {home_team} x {away_team} - {minute[-1]} minutos')
                    
                print('Finalizando o jogo')

                final_df = pd.DataFrame({
                    'date': date,
                    'minute': minute,
                    'match_id': match_id_list,
                    'homeTeam': homeTeam,
                    'awayTeam': awayTeam,
                    'goalHome': goalHome,
                    'goalAway': goalAway,
                    'shotsHome': shotsHome,
                    'shotsAway': shotsAway,
                    'crossesHome': crossesHome,
                    'crossesAway': crossesAway,
                    'assistsHome': assistsHome,
                    'assistsAway': assistsAway,
                    'blockedShotsHome': blockedShotsHome,
                    'blockedShotsAway': blockedShotsAway,
                    'league': liga,
                    'corners_home': corners_home,
                    'corners_away': corners_away,
                    'redcards_home': redcards_home,
                    'redcards_away': redcards_away,
                    'shotsOffgoal_home': shotsOffgoal_home,
                    'shotsOffgoal_away': shotsOffgoal_away,
                    'shotsOngoal_home': shotsOngoal_home,
                    'shotsOngoal_away': shotsOngoal_away,
                    'yellowcards_home': yellowcards_home,
                    'yellowcards_away': yellowcards_away,
                    'passes_home': passes_home,
                    'passes_away': passes_away,
                    'fouls_c_home': fouls_c_home,
                    'fouls_c_away': fouls_c_away,
                    'fouls_won_home': fouls_won_home,
                    'fouls_won_away': fouls_won_away,
                    'offsides_home': offsides_home,
                    'offsides_away': offsides_away,
                    'tackles_home': tackles_home,
                    'tackles_away': tackles_away,

                })


                final_df['result'] = 0
                
                # reordenar as colunas do DataFrame
                final_df = final_df[colunas_ordenadas]
                
                # Abrir o arquivo em modo de anexação
                print('Salvando o dataframe')
                with open('data_live_scores_full.csv', 'a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)

                    # Iterar sobre as linhas do DataFrame
                    for index, row in final_df.iterrows():
                        # Converter a linha em uma lista e escrevê-la no arquivo
                        writer.writerow(row.tolist())
                    
                
            # traceback
            except Exception as e:
                print(f'Erro no jogo: {home_team} x {away_team}')
                traceback.print_exc()

                print(e)
                continue
    print('Finalizando o driver')
    driver.quit()



In [136]:
main_function('2023-11-13') 

Verificando jogos da data: 2023-11-13
Existem 2 jogos para a liga Argentina - Liga Profesional Argentina
Valor atual do dragger: 1158
Novo valor do dragger: 1153
-------------------
-------------------
2023-11-13 - CA Sarmiento x Godoy Cruz Antonio Tomba - 90+4:21 minutos
Valor atual do dragger: 1152
Novo valor do dragger: 1147
-------------------
-------------------
2023-11-13 - CA Sarmiento x Godoy Cruz Antonio Tomba - 90+3:42 minutos
Valor atual do dragger: 1148
Novo valor do dragger: 1143
-------------------
-------------------
2023-11-13 - CA Sarmiento x Godoy Cruz Antonio Tomba - 90+3:02 minutos
Valor atual do dragger: 1142
Novo valor do dragger: 1137
-------------------
-------------------
2023-11-13 - CA Sarmiento x Godoy Cruz Antonio Tomba - 90+2:23 minutos
Valor atual do dragger: 1138
Novo valor do dragger: 1133
-------------------
-------------------
2023-11-13 - CA Sarmiento x Godoy Cruz Antonio Tomba - 90+1:44 minutos
Valor atual do dragger: 1132
Novo valor do dragger: 112

Traceback (most recent call last):
  File "C:\Users\Pichau\AppData\Local\Temp\ipykernel_9352\2813789648.py", line 331, in main_function
    final_df = pd.DataFrame({
  File "C:\Users\Pichau\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py", line 733, in __init__
    mgr = dict_to_mgr(data, index, columns, dtype=dtype, copy=copy, typ=manager)
  File "C:\Users\Pichau\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\construction.py", line 503, in dict_to_mgr
    return arrays_to_mgr(arrays, columns, index, dtype=dtype, typ=typ, consolidate=copy)
  File "C:\Users\Pichau\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\construction.py", line 114, in arrays_to_mgr
    index = _extract_index(arrays)
  File "C:\Users\Pichau\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\construction.py", line 677, in _extract_index
    raise ValueError("All arrays must be of the same length")
ValueError: All arrays must be of the 

Valor atual do dragger: 1158
Novo valor do dragger: 1153
-------------------
-------------------
2023-11-13 - Instituto AC Córdoba x CA Barracas Central - 90+6:20 minutos
Valor atual do dragger: 1152
Novo valor do dragger: 1147
-------------------
-------------------
2023-11-13 - Instituto AC Córdoba x CA Barracas Central - 90+5:41 minutos
Valor atual do dragger: 1148
Novo valor do dragger: 1143
-------------------
-------------------
2023-11-13 - Instituto AC Córdoba x CA Barracas Central - 90+5:01 minutos
Valor atual do dragger: 1142
Novo valor do dragger: 1137
-------------------
-------------------
2023-11-13 - Instituto AC Córdoba x CA Barracas Central - 90+4:21 minutos
Valor atual do dragger: 1138
Novo valor do dragger: 1133
-------------------
-------------------
2023-11-13 - Instituto AC Córdoba x CA Barracas Central - 90+3:42 minutos
Valor atual do dragger: 1132
Novo valor do dragger: 1127
-------------------
-------------------
2023-11-13 - Instituto AC Córdoba x CA Barracas 

KeyboardInterrupt: 

In [130]:
df = pd.read_csv('data_live_scores_full.csv', encoding='utf-8')

In [132]:
# deixar o dataframe vazio, apenas com as colunas
# df = df[0:0]
# df.to_csv('data_live_scores_full.csv', index=False, encoding='utf-8')

In [125]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [126]:
df.tail(10)

,date,minute,homeTeam,awayTeam,goalHome,goalAway,shotsHome,shotsAway,crossesHome,crossesAway,...,fouls_c_home,fouls_c_away,fouls_won_home,fouls_won_away,offsides_home,offsides_away,tackles_home,tackles_away,result,match_id
697,2023-11-13,6:26,CA Sarmiento,Godoy Cruz Antonio Tomba,0,0.0,1,1.0,0,0.0,...,1,1.0,1,1.0,1,0.0,0,0.0,0,2023-11-13-CA Sarmiento-Godoy Cruz Antonio Tomba
698,2023-11-13,5:47,CA Sarmiento,Godoy Cruz Antonio Tomba,0,0.0,1,1.0,0,0.0,...,1,1.0,1,1.0,0,0.0,0,0.0,0,2023-11-13-CA Sarmiento-Godoy Cruz Antonio Tomba
699,2023-11-13,5:08,CA Sarmiento,Godoy Cruz Antonio Tomba,0,0.0,1,1.0,0,0.0,...,1,0.0,0,1.0,0,0.0,0,0.0,0,2023-11-13-CA Sarmiento-Godoy Cruz Antonio Tomba
700,2023-11-13,4:28,CA Sarmiento,Godoy Cruz Antonio Tomba,0,0.0,1,0.0,0,0.0,...,1,0.0,0,1.0,0,0.0,0,0.0,0,2023-11-13-CA Sarmiento-Godoy Cruz Antonio Tomba
701,2023-11-13,3:49,CA Sarmiento,Godoy Cruz Antonio Tomba,0,0.0,1,0.0,0,0.0,...,1,0.0,0,1.0,0,0.0,0,0.0,0,2023-11-13-CA Sarmiento-Godoy Cruz Antonio Tomba
702,2023-11-13,3:10,CA Sarmiento,Godoy Cruz Antonio Tomba,0,0.0,1,0.0,0,0.0,...,1,0.0,0,1.0,0,0.0,0,0.0,0,2023-11-13-CA Sarmiento-Godoy Cruz Antonio Tomba
703,2023-11-13,2:31,CA Sarmiento,Godoy Cruz Antonio Tomba,0,0.0,1,0.0,0,0.0,...,1,0.0,0,1.0,0,0.0,0,0.0,0,2023-11-13-CA Sarmiento-Godoy Cruz Antonio Tomba
704,2023-11-13,1:52,CA Sarmiento,Godoy Cruz Antonio Tomba,0,0.0,1,0.0,0,0.0,...,1,0.0,0,1.0,0,0.0,0,0.0,0,2023-11-13-CA Sarmiento-Godoy Cruz Antonio Tomba
705,2023-11-13,1:12,CA Sarmiento,Godoy Cruz Antonio Tomba,0,0.0,1,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,2023-11-13-CA Sarmiento-Godoy Cruz Antonio Tomba
706,2023-11-13,0:33,CA Sarmiento,Godoy Cruz Antonio Tomba,0,0.0,1,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,2023-11-13-CA Sarmiento-Godoy Cruz Antonio Tomba


In [127]:
# df.to_csv('data_live_scores_full.csv', index=False, encoding='utf-8')